## Trabajo Práctico 1: Estimación espectral no-paramétrica
### Germán Carlos Bertachini, José Fresneda Sánchez, y Sofia Yanes Sanchez

In [105]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

# esta pagina te permite ver el contenido de los archivos: myhdf5.hdfgroup.org/
# los archivos estan organizados como un Python dictionary (se acesan con keys)
h1 = h5py.File("H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5", "r")
l1 = h5py.File("L-L1_GWOSC_4KHZ_R1-1126257415-4096.hdf5", "r")

In [101]:
strain_h1 = h1["strain"]["Strain"][()]
strain_l1 = l1["strain"]["Strain"][()]

# datos iguales pa h1 y l1
duration = h1["meta"]["Duration"][()]  # [s] tiempo grabado en los datos
num_datos = len(strain_h1)  # [-] numero total de datos grabados
frec_muestreo = num_datos/duration  # [Hz]

#### Estimando le espectro de potencia de ruido de H1 y L1

Periodograma

In [ ]:
# para ver la forma de los datos
plt.plot(strain_h1)
plt.show()
plt.plot(strain_l1)
plt.show()

In [104]:
# limpiando pa solo tener ruido
event_h1 = np.argmax(np.abs(strain_h1))  # se ve el pico en los datos de h1

# vamos a "overlay" una seccion de "false" donde no queremos los datos pa remove el evento grav.
evento = np.full(len(strain_h1), True)  # creando 
evento[int(event_h1 - 1.5 * frec_muestreo) : int(event_h1 + 1.5 * frec_muestreo)] = False  # tamano razonable donde no tenemos ningun leakage del evento en el ruido
ruido_h1 = strain_h1[evento]
ruido_l1 = strain_l1[evento]  # como estamos tomando un window del evento grande la diferencia de 10ms no importa x

In [107]:
N = 2**16  # pa un window length de 16s que es razonable (necesita justificacion mejor)

# esto es todo del profe
t = np.linspace(0, 1, N)
w_dc = sp.signal.windows.chebwin(N, 100)  

# como solo estamos windowing una seccion del ruido (no todo) tenemos q tomar un window
segment_h1 = ruido_h1[:N]  # primer 16s de ruido 
segment_l1 = ruido_l1[:N]

seg_h1_win = segment_h1 * w_dc
seg_l1_win = segment_l1 * w_dc

ruido_H1 = np.fft.fft(seg_h1_win)
ruido_L1 = np.fft.fft(seg_l1_win)

R_h1 = ruido_H1 * np.conj(ruido_H1)
R_l1 = ruido_L1 * np.conj(ruido_L1)

f = np.linspace(-1, 1, N)


In [ ]:
# aqui vemos 
plt.semilogy(f, np.fft.fftshift(np.abs(R_h1)))
plt.show()
plt.semilogy(f, np.fft.fftshift(np.abs(R_l1)))
plt.show()

Periodogram Smoothing

In [ ]:
# esto se tiene q hacer x

Periodogram Averaging

In [114]:
# creando arrays vacios para meter nuestros datos
R_avg_h1 = np.zeros(N)
R_avg_l1 = np.zeros(N)

for i in range(len(ruido_h1) // N):
    # encontramos todos los periodogramas de cada segment/window
    segment_h1 = ruido_h1[i*N : i*N+N]
    segment_l1 = ruido_l1[i*N : i*N+N]
    seg_h1_win = segment_h1 * w_dc
    seg_l1_win = segment_l1 * w_dc
    ruido_H1 = np.fft.fft(seg_h1_win)
    ruido_L1 = np.fft.fft(seg_l1_win)
    R_h1 = ruido_H1 * np.conj(ruido_H1)
    R_l1 = ruido_L1 * np.conj(ruido_L1)

    # metiendo el valor encontrado en el array 
    R_avg_h1 += np.abs(R_h1)
    R_avg_l1 += np.abs(R_l1)

R_avg_h1 /= (len(ruido_h1) // N)  # dividiendo por num de datos utilizado
R_avg_l1 /= (len(ruido_h1) // N)

In [ ]:
# aqui vemos que guaaaa el ruido es mucho menos x
plt.semilogy(f, np.fft.fftshift(R_avg_h1))
plt.show()
plt.semilogy(f, np.fft.fftshift(R_avg_l1))
plt.show()